In [26]:
filename = 'punchResults.txt'
def read_file_to_array(filename):
    with open(filename, 'r') as file:
        lines = [line.strip() for line in file]
    return lines

array_of_lines = read_file_to_array(filename)
db = []
for line in array_of_lines:
    row = line.split('|')
    temp = []
    for s in row[1][1:-1].split(' '):
      if len(s) > 0:
        temp.append(int(s))

    for s in row[2][1:-1].split(' '):
      if len(s) > 0:
        temp.append(int(s))

    for s in row[3][1:-1].split(' '):
      if len(s) > 0:
        temp.append(int(s))

    for s in row[4][1:-1].split(' '):
      if len(s) > 0:
        temp.append(int(s))
    if len(temp) == 16:
      db.append((row[0], temp))
print(db[0:5])



[('uppercut', [255, 252, 248, 255, 304, 271, 249, 181, 225, 223, 221, 222, 342, 338, 335, 310]), ('uppercut', [177, 166, 170, 169, 329, 298, 267, 209, 220, 208, 202, 198, 323, 321, 318, 297]), ('hook', [378, 329, 291, 259, 221, 203, 193, 194, 452, 437, 413, 366, 284, 272, 256, 240]), ('uppercut', [399, 393, 391, 386, 253, 219, 216, 150, 413, 414, 415, 414, 330, 308, 303, 278]), ('hook', [218, 218, 218, 217, 273, 274, 276, 276, 219, 219, 219, 218, 327, 327, 329, 329])]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class PunchDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.classes = list(set(item[0] for item in data))
        print(len(self.classes))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        punch_type, features = self.data[idx]
        label = self.class_to_idx[punch_type]
        return torch.tensor(features, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

class PunchClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(PunchClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

data = db

dataset = PunchDataset(data)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

input_size = 16
hidden_size = 32
num_classes = len(dataset.classes)
model = PunchClassifier(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in dataloader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

sample = torch.tensor([255, 252, 248, 255, 304, 271, 249, 181, 225, 223, 221, 222, 342, 338, 335, 310], dtype=torch.float32)
prediction = model(sample)
predicted_class = dataset.classes[torch.argmax(prediction).item()]
print(f'Predicted punch type: {predicted_class}')

In [55]:
import random

for i in range(len(db)):
  index = random.randint(0, len(db)-1)
  temp = db[index][1]
  sample = torch.tensor(temp, dtype=torch.float32)
  prediction = model(sample)
  predicted_class = dataset.classes[torch.argmax(prediction).item()]
  if predicted_class != db[index][0]:
    print("fail", db[index][0])
  # print(f'Predicted punch type: {predicted_class}')
  # print(f'Real punch type: {db[index][0]}')

fail jab
fail hook
fail uppercut
fail hook
fail uppercut
fail jab
fail jab
fail jab
fail hook
fail jab
fail jab
fail jab


In [58]:

torch.save(model.state_dict(), 'punch_classifier.pth')

In [90]:

loaded_model = PunchClassifier(input_size, hidden_size, num_classes)
loaded_model.load_state_dict(torch.load('punch_classifier.pth'))
loaded_model.eval()

<ipython-input-90-b91b74c6e21c>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model.load_state_dict(torch.load('punch_classifier.pth'))


PunchClassifier(
  (fc1): Linear(in_features=16, out_features=32, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=32, out_features=3, bias=True)
)

In [93]:
print(input_size, hidden_size, num_classes)

16 32 3


In [92]:
for i in range(len(db)):
  index = random.randint(0, len(db)-1)
  temp = db[index][1]
  sample = torch.tensor(temp, dtype=torch.float32)
  prediction = loaded_model(sample)
  predicted_class = dataset.classes[torch.argmax(prediction).item()]
  if predicted_class != db[index][0]:
    print("fail", db[index][0])

fail jab
fail jab
fail hook
fail uppercut
fail hook
fail uppercut
fail jab
fail uppercut
fail jab
fail hook
fail hook
fail hook
fail jab
fail hook
fail hook
fail jab
fail jab
fail jab
fail uppercut
fail uppercut
fail hook
fail jab


In [94]:
print(dataset.classes)

['hook', 'uppercut', 'jab']
